In [ ]:
# Check core SDK version number
import azureml.core
from azureml.core import Experiment

print("SDK version:", azureml.core.VERSION)


In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

In [ ]:
from azureml.core import Datastore, Dataset

ds = Datastore(ws, "generalpurposeaccount")

In [ ]:
delta_file = Dataset.File.from_files(path = [(ds, '')])
data_folder = delta_file.as_mount()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster" # "v100-1-g-1-n"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Compute target not found...')

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
from azureml.core.environment import Environment

tf_env = Environment.get(ws, name='AzureML-TensorFlow-2.0-GPU')
tf_env_c = tf_env.clone("deltalake")

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

conda_dep = CondaDependencies()

# Installs deltalake pandas package
conda_dep.add_pip_package("pandas")
conda_dep.add_pip_package("deltalake")

# Adds dependencies to PythonSection of myenv
tf_env_c.python.conda_dependencies=conda_dep

In [ ]:
from azureml.core import ScriptRunConfig

script_folder = './deltalake'

src = ScriptRunConfig(source_directory=script_folder,
                      script='read_deltalake.py',
                      arguments=['--data-folder', data_folder],
                      compute_target=compute_target,
                      environment=tf_env_c)

In [ ]:
exp = Experiment(workspace=ws, name='delta-read')

run = exp.submit(src)
run